In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# Since our data is comprised of images, we have to load each image from each directory and store them into arrays


# This function returns all the filenames in the chosen directory
def get_files(directory):
    files = []
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    return files

train = []
labels = []

for file in get_files('/kaggle/input/campqmind-hackathon-2022/train/'):
    img = cv2.imread(file)
    img_array = np.asarray(img)
    

    if img_array.shape != (102, 136, 3):
        continue

    
    converted = tf.image.rgb_to_grayscale(img_array)
    
    train.append(converted)
    labels.append(file.split('/')[-2])

test = []
for file in get_files('/kaggle/input/campqmind-hackathon-2022/test/'):
    img = cv2.imread(file)
    img_array = np.asarray(img)
    test.append(img_array)
    

print(len(train), len(labels))

# We have to use a plotting function and cv2 to show the image from the array data


10498 10498


In [5]:
from sklearn.preprocessing import OneHotEncoder

'''
enc = OneHotEncoder(handle_unknown='ignore')
labels = np.array(labels).reshape(-1,1)
enc.fit(labels)
hot = enc.transform(labels).toarray()
'''

# OR 

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(labels)
lab = le.transform(labels)

print(lab)
print(len(lab))
print(len(hot))





[2 2 2 ... 0 0 0]
10498


NameError: name 'hot' is not defined

# Loading Data in

In [6]:
os.listdir('/kaggle')

['lib', 'input', 'working']

In [7]:
os.listdir('/kaggle/input/campqmind-hackathon-2022')

['sample_submission.csv', 'test', 'train']

In [9]:

from sklearn.model_selection import train_test_split

train = np.array(train)
train_data, test_data, train_labels, test_labels = train_test_split(train, lab, test_size=0.33, random_state=42)


print(len(train_data), len(train_labels))
print(len(test_data), len(test_labels))

print(test_labels)

7033 7033
3465 3465
[1 0 0 ... 2 2 0]


In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(102, 136, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 100, 134, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 67, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 48, 65, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 32, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 22, 30, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
Layer (type)                

In [16]:


test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

109/109 - 6s - loss: 0.3577 - accuracy: 0.9180


# Creating a Submission

In [ ]:
# creating a dataframe to submit
# 'Id' should have the image names, 'Categories' should have the name of the class
submisson = pd.DataFrame(0, index=np.arange(0, 4500), columns=['Id', 'Categories'])
# create a csv
submisson.to_csv('/kaggle/working/submission.csv', index=False)